In [1]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import pandas as pd
import numpy as np

import optuna
from objective import Objective

#### 再現のための乱数値指定

In [2]:
SEED = 334

#### 学習データの読み込み

In [3]:
boston = load_boston()
df_X = pd.DataFrame(boston.data, columns = boston.feature_names, dtype = float)
df_y = pd.Series(boston.target, name = 'PRICE', dtype = float)

#### データの分割

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size = 0.2, random_state = SEED)

#### インスタンス化

In [5]:
objective = Objective(RandomForestRegressor, X_train, y_train, random_state = SEED, cv = 5)

#### 最適化

In [6]:
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials=10)

[I 2020-07-28 11:47:50,718] Finished trial#0 with value: 0.8453904683037697 with parameters: {'min_samples_split': 2, 'max_depth': 496, 'n_estimators': 191}. Best is trial#0 with value: 0.8453904683037697.
[I 2020-07-28 11:47:52,065] Finished trial#1 with value: 0.8414318123870709 with parameters: {'min_samples_split': 5, 'max_depth': 248, 'n_estimators': 145}. Best is trial#0 with value: 0.8453904683037697.
[I 2020-07-28 11:47:56,383] Finished trial#2 with value: 0.8419398754491342 with parameters: {'min_samples_split': 8, 'max_depth': 371, 'n_estimators': 499}. Best is trial#0 with value: 0.8453904683037697.
[I 2020-07-28 11:48:00,607] Finished trial#3 with value: 0.844834986501672 with parameters: {'min_samples_split': 4, 'max_depth': 188, 'n_estimators': 446}. Best is trial#0 with value: 0.8453904683037697.
[I 2020-07-28 11:48:01,398] Finished trial#4 with value: 0.8374219516359276 with parameters: {'min_samples_split': 12, 'max_depth': 53, 'n_estimators': 90}. Best is trial#0 with

#### 最適モデルを得る

In [7]:
best_clf = objective.clf(**study.best_params, **objective.fixed_params)

#### スコアで性能評価

In [8]:
best_clf.fit(X_train, y_train)
y_pred_on_test = best_clf.predict(X_test)
print(r2_score(y_test, y_pred_on_test))

0.8655367230685075
